## Chapter 8. Type Hints in Functions

In [1]:
# Example 8-1. show_count from messages.py without type hints

def show_count(count, word):
    if count == 1:
        return f'1 {word}'
    count_str = str(count) if count else 'no'
    return f'{count_str} {word}s'

In [4]:
print(show_count(99, 'bird'))
print(show_count(1, 'bird'))
print(show_count(0, 'bird'))

99 birds
1 bird
no birds


In [3]:
# Example 8-3. showcount from hints_2/messages.py with an optional parameter

def show_count(count: int, singular: str, plural: str = '') -> str:
    if count == 1:
        return f'1 {singular}'
    count_str = str(count) if count else 'no'
    if not plural:
        plural = singular + 's'
    return f'{count_str} {plural}'

In [ ]:
from typing import Optional

# Use None instead of '' for str default
def show_count(count: int, singular: str, plural: Optional[str] = None) -> str:
    if count == 1:
        return f'1 {singular}'
    count_str = str(count) if count else 'no'
    if not plural:
        plural = singular + 's'
    return f'{count_str} {plural}'

In [5]:
# Example 8-4 is a silly example that contrasts duck typing and nominal typing, as well
# as static type checking and runtime behavior.6

class Bird:
    pass

class Duck(Bird):
    def quack(self):
        print('Quack!')

def alert(birdie):
    birdie.quack()

def alert_duck(birdie: Duck) -> None:
    birdie.quack()

def alert_bird(birdie: Bird) -> None:  # Wrong type annotation
    birdie.quack()

In [6]:
# Example 8-5. daffy.py

daffy = Duck()
alert(daffy)
alert_duck(daffy)
alert_bird(daffy)

Quack!
Quack!
Quack!


In [7]:
# Example 8-6. woody.py

woody = Bird()
alert(woody)
alert_duck(woody)
alert_bird(woody)

AttributeError: 'Bird' object has no attribute 'quack'

In [1]:
# Example 8-8. tokenize with type hints for Python ≥ 3.9

def tokenize(text: str) -> list[str]:
    return text.upper().split()

In [3]:
# Example 8-11. coordinates.py with the geohash function

from geolib import geohash as gh # type: ignore

PRECISION = 9

def geohash(lat_lon: tuple[float, float]) -> str:
    return gh.encode(*lat_lon, PRECISION)

In [4]:
# Example 8-12. coordinates_named.py with the NamedTuple Coordinates and the geo hash function
from typing import NamedTuple
from geolib import geohash as gh # type: ignore

PRECISION = 9

class Coordinate(NamedTuple):
    lat: float
    lon: float

def geohash(lat_lon: Coordinate) -> str:
    return gh.encode(*lat_lon, PRECISION)

In [5]:
# Example 8-13. columnize.py returns a list of tuples of strings

from collections.abc import Sequence

def columnize(
sequence: Sequence[str], num_columns: int = 0
) -> list[tuple[str, ...]]:
    if num_columns == 0:
        num_columns = round(len(sequence) ** 0.5)
    num_rows, reminder = divmod(len(sequence), num_columns)
    num_rows += bool(reminder)
    return [tuple(sequence[i::num_rows]) for i in range(num_rows)]

### Iterable

The typing.List documentation I just quoted recommends Sequence and Iterable
for function parameter type hints.

In [7]:
# Example 8-15. replacer.py

from collections.abc import Iterable

FromTo = tuple[str, str]

def zip_replace(text: str, changes: Iterable[FromTo]) -> str:
    for from_, to in changes:
        text = text.replace(from_, to)
    return text

In [8]:
l33t = [('a', '4'), ('e', '3'), ('i', '1'), ('o', '0')]
text = 'mad skilled noob powned leet'
zip_replace(text, l33t)

'm4d sk1ll3d n00b p0wn3d l33t'

In [9]:
# Parameterized Generics and TypeVar
# Example 8-16. sample.py

from collections.abc import Sequence
from random import shuffle
from typing import TypeVar

T = TypeVar('T')

def sample(population: Sequence[T], size: int) -> list[T]:
    if size < 1:
        raise ValueError('size must be >= 1')
    result = list(population)
    shuffle(result)
    return result[:size]

In [10]:
# Example 8-17. mode_float.py: mode that operates on float and subtypes13
from collections import Counter
from collections.abc import Iterable

def mode(data: Iterable[float]) -> float:
    pairs = Counter(data).most_common(1)
    if len(pairs) == 0:
        raise ValueError('no mode for empty data')
    return pairs[0][0]

In [ ]:
# Example 8-18. mode_hashable.py: same as Example 8-17, with a more flexible signature
from collections import Counter
from collections.abc import Iterable, Hashable
from typing import TypeVar

HashableT = TypeVar('HashableT', bound=Hashable)

def mode(data: Iterable[HashableT]) -> HashableT:
    pairs = Counter(data).most_common(1)
    if len(pairs) == 0:
        raise ValueError('no mode for empty data')
    return pairs[0][0]

### Static Protocol

The Protocol type, as presented in PEP 544—Protocols: Structural subtyping (static
duck typing), is similar to interfaces in Go: a protocol type is defined by specifying
one or more methods, and the type checker verifies that those methods are implemented
where that protocol type is required.


In [11]:
# Example 8-19. top function with an undefined T type parameter
def top(series: Iterable[T], length: int) -> list[T]:
    ordered = sorted(series, reverse=True)
    return ordered[:length]

In [25]:
print(top([4, 1, 5, 2, 6, 7, 3], 3))

l = 'mango pear apple kiwi banana'.split()
print(top(l, 3))

l2 = [(len(s), s) for s in l]
print(l2)
print(top(l2, 3))

# Error since object() does not have the key for sorted()
obj_no_sort_key = [object() for _ in range(4)]
print(obj_no_sort_key)
print(top(obj_no_sort_key, 2))

[7, 6, 5]
['pear', 'mango', 'kiwi']
[(5, 'mango'), (4, 'pear'), (5, 'apple'), (4, 'kiwi'), (6, 'banana')]
[(6, 'banana'), (5, 'mango'), (5, 'apple')]
[<object object at 0x0000027853188C30>, <object object at 0x0000027853188C70>, <object object at 0x0000027853188C80>, <object object at 0x0000027853188C90>]


TypeError: '<' not supported between instances of 'object' and 'object'

In [23]:
# Example 8-20. comparable.py: definition of a SupportsLessThan Protocol type
from typing import Protocol, Any

class SupportsLessThan(Protocol):
    def __lt__(self, other: Any) -> bool: ...

In [24]:
# Example 8-21. top.py: definition of the top function using a TypeVar with bound=SupportsLessThan
from collections.abc import Iterable
from typing import TypeVar
# from comparable import SupportsLessThan

LT = TypeVar('LT', bound=SupportsLessThan)
def top(series: Iterable[LT], length: int) -> list[LT]:
    ordered = sorted(series, reverse=True)
    return ordered[:length]

In [28]:
# Example 8-22. top_test.py: partial listing of the test suite for top
from collections.abc import Iterator
from typing import TYPE_CHECKING
import pytest
# from top import top

# several lines omitted
def test_top_tuples() -> None:
    fruit = 'mango pear apple kiwi banana'.split()
    series: Iterator[tuple[int, str]] = (
    (len(s), s) for s in fruit)
    length = 3
    expected = [(6, 'banana'), (5, 'mango'), (5, 'apple')]
    result = top(series, length)
    if TYPE_CHECKING:
        reveal_type(series)
        reveal_type(expected)
        reveal_type(result)
    assert result == expected

# intentional type error
def test_top_objects_error() -> None:
    series = [object() for _ in range(4)]
    if TYPE_CHECKING:
        reveal_type(series)
    with pytest.raises(TypeError) as excinfo:
        top(series, 3)
    assert "'<' not supported" in str(excinfo.value)

In [29]:
test_top_tuples()
test_top_objects_error()

In [32]:
# Callable
# Example 8-24. Illustrating variance.
from collections.abc import Callable

def update(
    probe: Callable[[], float],         # Callable type: no arguments and returns a float
    display: Callable[[float], None]    # Callable type: float argument and returns None
) -> None:
    temperature = probe()
    # imagine lots of control code here
    display(temperature)

def probe_ok() -> int:
    return 42

def display_wrong(temperature: int) -> None:
    print(hex(temperature))


def display_ok(temperature: complex) -> None:
    print(temperature)


In [31]:
update(probe_ok, display_ok) # OK
update(probe_ok, display_wrong) # type error in type check tool (Mypy)

42
0x2a


In [2]:
# Example 7-9. tag generates HTML; a keyword-only argument cls is used to pass
# “class” attributes as a workaround because class is a keyword in Python

def tag(name, *content, cls=None, **attrs):
    """Generate one or more HTML tags"""
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                           for attr, value
                           in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' %
                         (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [ ]:
# Type annotated version of tag

from typing import Optional

def tag(__name: str, *content: str, class_: Optional[str] = None, **attrs: str) -> str:
    pass